In [2]:
import os
import re
import json
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import xml.etree.ElementTree as et
import io
import glob
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

#tqdm_notebook.pandas()

In [63]:
xml_file = "./new_data/combined_xml/NCT00058058.xml"
tree = et.parse(xml_file)
root = tree.getroot()
col_names = ['nct_id', 'brief_title', 'official_title', 'overall_status', 
         'start_date','completion_date', 'phase', 'study_type', 
             'brief_summary', 'detailed_description', 'enrollment',
             'condition', 'intervention_name', 'eligibility']

name1 = ['nct_id','brief_title', 'official_title', 'overall_status', 
         'start_date','completion_date', 'phase', 'study_type', 'eligibility']

name2 = ['brief_summary', 'detailed_description']
             
name3 = ['enrollment', 'condition']

df = pd.DataFrame(columns = col_names)
row = {name: ''for name in col_names} 

for d in tree.iter():
    for txt in d.iter():
        if d.tag in name1:
            row[d.tag] = d.text
        if d.tag in name2:
            row[d.tag] = d.getchildren()[0].text
        if d.tag in name3:
            row[d.tag] = txt.text
        if d.tag == "intervention_name":
            intSearch = root.find('intervention_browse')
            if intSearch:
                row[d.tag]= [i.text for i in intSearch.findall('mesh_term')]
            else:
                row[d.tag] = [i.text for i in root.iter('intervention_name')]
        if d.tag =='eligibility':
            row[d.tag] = root.find('eligibility').find('criteria').find('textblock').text
df = df.append(pd.Series(row, index = col_names), ignore_index = True)

for d in tree.iter():
    if d.tag == "intervention_name":
        intSearch = root.find('intervention_browse')
        if intSearch:
            print([i.text for i in intSearch.findall('mesh_term')])
        else:
            print([i.text for i in root.iter('intervention_name')])
    #print(row[d.tag] == "intervention_name")

# for fc in root:
#     for sc in fc:
#         if sc.tag in name1:
#             print(sc.text) 
#             row[fc.tag] = fc.text
#         if fc.tag in name2:
#             row[fc.tag] = d.getchildren()[0].text
#         if fc.tag in name3:
#             row[fc.tag] = sc.text
# df = df.append(pd.Series(row, index = col_names), ignore_index = True)

# for d in tree.iter():
#     for txt in d.iter():
#         if d.tag in name3:
#             print(txt.text)

['MRI']


In [64]:
#df.head()
df['intervention_name'] = df['intervention_name'].apply(', '.join)
df.head()

,nct_id,brief_title,official_title,overall_status,start_date,completion_date,phase,study_type,brief_summary,detailed_description,enrollment,condition,intervention_name,eligibility
0,NCT00058058,Magnetic Resonance Imaging in Women Recently D...,MRI Evaluation Of The Contralateral Breast In ...,Completed,February 2003,August 2008,N/A,Interventional,\n RATIONALE: Diagnostic procedures such ...,\n OBJECTIVES:\n\n - Determine th...,1007,Contralateral Breast Cancer,MRI,\n DISEASE CHARACTERISTICS:\n\n ...


In [61]:
df.intervention_name[0]

'MRI'

In [9]:
xml_file = "./new_data/combined_xml/NCT00000500.xml"
tree = et.parse(xml_file)
root = tree.getroot()
#res = []
#for t in root: 
col_names = ['nct_id', 'brief_title', 'official_title', 'overall_status', 
         'start_date','completion_date', 'phase', 'study_type', 
             'brief_summary', 'detailed_description', 
             'intervention','condition', 'enrollment']

name1 = ['nct_id','brief_title', 'official_title', 'overall_status', 
         'start_date','completion_date', 'phase', 'study_type']

name2 = ['brief_summary', 'detailed_description']
             
name3 = ['intervention','condition', 'enrollment']

df = pd.DataFrame(columns = name1)
row = {name: ''for name in name1} 
for d in tree.iter():
    #for txt in d.iter():
    if d.tag =='enrollment':
        print(d)

#         if d.tag in name2:
#             row[d.tag] = d.getchildren()[0].text
#         if d.tag in name3:
#             row[d.tag] = txt.text
# df = df.append(pd.Series(row, index = col_names), ignore_index = True)

In [2]:
filePath = glob.glob("./new_data/combined_xml/*.xml")
col_names = ['nct_id', 'brief_title', 'official_title', 'overall_status', 
         'start_date','completion_date', 'phase', 'study_type', 
             'brief_summary', 'detailed_description', 
             'intervention','condition', 'enrollment']

name1 = ['nct_id','brief_title', 'official_title', 'overall_status', 
         'start_date','completion_date', 'phase', 'study_type']

name2 = ['brief_summary', 'detailed_description']

name3 = ['intervention','condition', 'enrollment']

df = pd.DataFrame(columns = col_names)
bad_file = []

for files in tqdm(filePath):
    try:
        with open(files, 'r', encoding="utf-8") as content:
            tree = et.parse(content)
            root = tree.getroot()

            row = {name: ''for name in col_names} 

            for d in tree.iter():
                for txt in d.iter():
                    if d.tag in name1:
                        row[d.tag] = d.text
                    if d.tag in name2:
                        row[d.tag] = d.getchildren()[0].text
                    if d.tag in name3:
                        row[d.tag] = txt.text
            df = df.append(pd.Series(row, index = col_names), ignore_index = True)
    except:
        bad_file.append(files)


In [3]:
print(len(df))
df.head()

61781


,nct_id,brief_title,official_title,overall_status,start_date,completion_date,phase,study_type,brief_summary,detailed_description,intervention,condition,enrollment
0,NCT00391586,Erlotinib and Standard Platinum-Based Chemothe...,INST 0601C: A Non-Randomized Phase II Protocol...,Terminated,July 2006,May 2012,Phase 2,Interventional,\n This study was conducted to compare th...,\n To compare the activities (the progres...,gemcitabine + platinum,"Carcinoma, Non-Small-Cell Lung",45
1,NCT00058058,Magnetic Resonance Imaging in Women Recently D...,MRI Evaluation Of The Contralateral Breast In ...,Completed,February 2003,August 2008,N/A,Interventional,\n RATIONALE: Diagnostic procedures such ...,\n OBJECTIVES:\n\n - Determine th...,magnetic resonance imaging,Contralateral Breast Cancer,1007
2,NCT00038974,Hepatitis C Antiviral Resistance in African-Am...,Study of Viral Resistance to Antiviral Therapy...,Completed,August 2002,November 2005,Phase 3,Interventional,\n This study is designed to test the hyp...,,Interferon and rivavirin,Hepatitis C,401
3,NCT01473303,Combination Chemotherapy With or Without Ganit...,Phase IB/Randomized Phase II Study of Folfirin...,Withdrawn,August 2012,,Phase 1/Phase 2,Interventional,"\n RATIONALE: Drugs used in chemotherapy,...",\n OBJECTIVES:\n\n Primary\n\n ...,Arm II,Pancreatic Cancer,0
4,NCT02528643,A Study to Assess the Efficacy and Safety of E...,"A Phase 2, Randomized, Double-Blind, Placebo-C...","Active, not recruiting","November 9, 2015",May 2019,Phase 2,Interventional,\n The purpose of the study was to evalua...,,Placebo,Advanced Hepatocellular Carcinoma,165


In [4]:
df.to_csv('cancerTrials.csv', index=False)

In [5]:
bad_file

[]

In [7]:
df.tail()

,nct_id,brief_title,official_title,overall_status,start_date,completion_date,phase,study_type,brief_summary,detailed_description,intervention,condition,enrollment
0,NCT01473303,Combination Chemotherapy With or Without Ganit...,Phase IB/Randomized Phase II Study of Folfirin...,Withdrawn,August 2012,,Phase 1/Phase 2,Interventional,"\n RATIONALE: Drugs used in chemotherapy,...",\n OBJECTIVES:\n\n Primary\n\n ...,Arm II,Pancreatic Cancer,0


In [8]:
filePath = glob.glob("./new_data/combined_xml/NCT*.xml")
for files in filePath:
    print(files)
     #with open(files, 'r', encoding="utf-8") as content:

./new_data/combined_xml/NCT00391586.xml
./new_data/combined_xml/NCT00058058.xml
./new_data/combined_xml/NCT00038974.xml
./new_data/combined_xml/NCT01473303.xml
./new_data/combined_xml/NCT02528643.xml
./new_data/combined_xml/NCT01435356.xml
./new_data/combined_xml/NCT01837862.xml
./new_data/combined_xml/NCT02181140.xml
./new_data/combined_xml/NCT01902173.xml
./new_data/combined_xml/NCT02568267.xml
./new_data/combined_xml/NCT02840162.xml
./new_data/combined_xml/NCT03180177.xml
./new_data/combined_xml/NCT03857737.xml
./new_data/combined_xml/NCT03228043.xml
./new_data/combined_xml/NCT00036959.xml
./new_data/combined_xml/NCT02518958.xml
./new_data/combined_xml/NCT02759016.xml
./new_data/combined_xml/NCT01794520.xml
./new_data/combined_xml/NCT02386683.xml
./new_data/combined_xml/NCT00335556.xml
./new_data/combined_xml/NCT00277017.xml
./new_data/combined_xml/NCT03102606.xml
./new_data/combined_xml/NCT00004055.xml
./new_data/combined_xml/NCT00348595.xml
./new_data/combined_xml/NCT01967927.xml


./new_data/combined_xml/NCT01556009.xml
./new_data/combined_xml/NCT01455272.xml
./new_data/combined_xml/NCT02269280.xml
./new_data/combined_xml/NCT01716481.xml
./new_data/combined_xml/NCT03153514.xml
./new_data/combined_xml/NCT00145626.xml
./new_data/combined_xml/NCT03047213.xml
./new_data/combined_xml/NCT02537158.xml
./new_data/combined_xml/NCT00604435.xml
./new_data/combined_xml/NCT02268175.xml
./new_data/combined_xml/NCT02511821.xml
./new_data/combined_xml/NCT00084708.xml
./new_data/combined_xml/NCT00635830.xml
./new_data/combined_xml/NCT01741532.xml
./new_data/combined_xml/NCT02579551.xml
./new_data/combined_xml/NCT00006079.xml
./new_data/combined_xml/NCT02498977.xml
./new_data/combined_xml/NCT01957007.xml
./new_data/combined_xml/NCT00597597.xml
./new_data/combined_xml/NCT00983619.xml
./new_data/combined_xml/NCT03780517.xml
./new_data/combined_xml/NCT03476239.xml
./new_data/combined_xml/NCT00774878.xml
./new_data/combined_xml/NCT00961844.xml
./new_data/combined_xml/NCT01898104.xml


./new_data/combined_xml/NCT02686463.xml
./new_data/combined_xml/NCT00399867.xml
./new_data/combined_xml/NCT02027948.xml
./new_data/combined_xml/NCT01181375.xml
./new_data/combined_xml/NCT01374620.xml
./new_data/combined_xml/NCT01604642.xml
./new_data/combined_xml/NCT00084656.xml
./new_data/combined_xml/NCT00094445.xml
./new_data/combined_xml/NCT03059173.xml
./new_data/combined_xml/NCT03076190.xml
./new_data/combined_xml/NCT03690128.xml
./new_data/combined_xml/NCT02849145.xml
./new_data/combined_xml/NCT01230710.xml
./new_data/combined_xml/NCT00006127.xml
./new_data/combined_xml/NCT03711422.xml
./new_data/combined_xml/NCT02869295.xml
./new_data/combined_xml/NCT03322579.xml
./new_data/combined_xml/NCT03059615.xml
./new_data/combined_xml/NCT01422629.xml
./new_data/combined_xml/NCT02593968.xml
./new_data/combined_xml/NCT01305967.xml
./new_data/combined_xml/NCT00883753.xml
./new_data/combined_xml/NCT02810756.xml
./new_data/combined_xml/NCT01748448.xml
./new_data/combined_xml/NCT00462124.xml


./new_data/combined_xml/NCT02621541.xml
./new_data/combined_xml/NCT00979576.xml
./new_data/combined_xml/NCT01872078.xml
./new_data/combined_xml/NCT02063529.xml
./new_data/combined_xml/NCT00608972.xml
./new_data/combined_xml/NCT00957853.xml
./new_data/combined_xml/NCT01320852.xml
./new_data/combined_xml/NCT00626457.xml
./new_data/combined_xml/NCT01482702.xml
./new_data/combined_xml/NCT00306202.xml
./new_data/combined_xml/NCT01203735.xml
./new_data/combined_xml/NCT02631746.xml
./new_data/combined_xml/NCT00939146.xml
./new_data/combined_xml/NCT03722407.xml
./new_data/combined_xml/NCT00053287.xml
./new_data/combined_xml/NCT00025311.xml
./new_data/combined_xml/NCT03814590.xml
./new_data/combined_xml/NCT00346632.xml
./new_data/combined_xml/NCT00447005.xml
./new_data/combined_xml/NCT00204373.xml
./new_data/combined_xml/NCT03137537.xml
./new_data/combined_xml/NCT00002976.xml
./new_data/combined_xml/NCT00265941.xml
./new_data/combined_xml/NCT01573923.xml
./new_data/combined_xml/NCT03448666.xml


./new_data/combined_xml/NCT00664261.xml
./new_data/combined_xml/NCT02582125.xml
./new_data/combined_xml/NCT03293628.xml
./new_data/combined_xml/NCT01022346.xml
./new_data/combined_xml/NCT00286013.xml
./new_data/combined_xml/NCT00004916.xml
./new_data/combined_xml/NCT00723658.xml
./new_data/combined_xml/NCT01450241.xml
./new_data/combined_xml/NCT01116791.xml
./new_data/combined_xml/NCT03316144.xml
./new_data/combined_xml/NCT01976169.xml
./new_data/combined_xml/NCT00979173.xml
./new_data/combined_xml/NCT03813836.xml
./new_data/combined_xml/NCT01725100.xml
./new_data/combined_xml/NCT00384176.xml
./new_data/combined_xml/NCT01596400.xml
./new_data/combined_xml/NCT01492114.xml
./new_data/combined_xml/NCT02765074.xml
./new_data/combined_xml/NCT00357448.xml
./new_data/combined_xml/NCT01110486.xml
./new_data/combined_xml/NCT01456156.xml
./new_data/combined_xml/NCT00065442.xml
./new_data/combined_xml/NCT03213028.xml
./new_data/combined_xml/NCT02371590.xml
./new_data/combined_xml/NCT03319511.xml


...

./new_data/combined_xml/NCT00679211.xml
./new_data/combined_xml/NCT01639508.xml
./new_data/combined_xml/NCT00033631.xml
./new_data/combined_xml/NCT01763606.xml
./new_data/combined_xml/NCT00337389.xml
./new_data/combined_xml/NCT01255631.xml
./new_data/combined_xml/NCT00297895.xml
./new_data/combined_xml/NCT03150836.xml
./new_data/combined_xml/NCT00359450.xml
./new_data/combined_xml/NCT00524810.xml
./new_data/combined_xml/NCT03144843.xml
./new_data/combined_xml/NCT01174121.xml
./new_data/combined_xml/NCT02549261.xml
./new_data/combined_xml/NCT03537482.xml
./new_data/combined_xml/NCT03250130.xml
./new_data/combined_xml/NCT02114385.xml
./new_data/combined_xml/NCT00652080.xml
./new_data/combined_xml/NCT03445000.xml
./new_data/combined_xml/NCT00816595.xml
./new_data/combined_xml/NCT00083694.xml
./new_data/combined_xml/NCT02026323.xml
./new_data/combined_xml/NCT02402062.xml
./new_data/combined_xml/NCT01530594.xml
./new_data/combined_xml/NCT02583568.xml
./new_data/combined_xml/NCT00343408.xml


./new_data/combined_xml/NCT01478477.xml
./new_data/combined_xml/NCT01158222.xml
./new_data/combined_xml/NCT00238121.xml
./new_data/combined_xml/NCT02017717.xml
./new_data/combined_xml/NCT02208843.xml
./new_data/combined_xml/NCT01200797.xml
./new_data/combined_xml/NCT00241384.xml
./new_data/combined_xml/NCT00727584.xml
./new_data/combined_xml/NCT03913026.xml
./new_data/combined_xml/NCT00623090.xml
./new_data/combined_xml/NCT00228332.xml
./new_data/combined_xml/NCT02115672.xml
./new_data/combined_xml/NCT00579332.xml
./new_data/combined_xml/NCT00056654.xml
./new_data/combined_xml/NCT02364999.xml
./new_data/combined_xml/NCT00413699.xml
./new_data/combined_xml/NCT02975934.xml
./new_data/combined_xml/NCT02230462.xml
./new_data/combined_xml/NCT00871481.xml
./new_data/combined_xml/NCT00688415.xml
./new_data/combined_xml/NCT00434226.xml
./new_data/combined_xml/NCT02188745.xml
./new_data/combined_xml/NCT02335268.xml
./new_data/combined_xml/NCT00708201.xml
./new_data/combined_xml/NCT02740400.xml


./new_data/combined_xml/NCT00618826.xml
./new_data/combined_xml/NCT00127127.xml
./new_data/combined_xml/NCT02833155.xml
./new_data/combined_xml/NCT02959398.xml
./new_data/combined_xml/NCT00447356.xml
./new_data/combined_xml/NCT00101686.xml
./new_data/combined_xml/NCT02676986.xml
./new_data/combined_xml/NCT01419665.xml
./new_data/combined_xml/NCT03431948.xml
./new_data/combined_xml/NCT03732547.xml
./new_data/combined_xml/NCT00299117.xml
./new_data/combined_xml/NCT01627041.xml
./new_data/combined_xml/NCT00587600.xml
./new_data/combined_xml/NCT03023605.xml
./new_data/combined_xml/NCT00967369.xml
./new_data/combined_xml/NCT00004932.xml
./new_data/combined_xml/NCT00458510.xml
./new_data/combined_xml/NCT00003085.xml
./new_data/combined_xml/NCT01468922.xml
./new_data/combined_xml/NCT02994498.xml
./new_data/combined_xml/NCT03598270.xml
./new_data/combined_xml/NCT00333840.xml
./new_data/combined_xml/NCT03389126.xml
./new_data/combined_xml/NCT02868151.xml
./new_data/combined_xml/NCT02390986.xml


./new_data/combined_xml/NCT02494167.xml
./new_data/combined_xml/NCT02887586.xml
./new_data/combined_xml/NCT03724253.xml
./new_data/combined_xml/NCT03167125.xml
./new_data/combined_xml/NCT02590315.xml
./new_data/combined_xml/NCT00128362.xml
./new_data/combined_xml/NCT00003695.xml
./new_data/combined_xml/NCT01525940.xml
./new_data/combined_xml/NCT03203369.xml
./new_data/combined_xml/NCT03141684.xml
./new_data/combined_xml/NCT03138421.xml
./new_data/combined_xml/NCT03780283.xml
./new_data/combined_xml/NCT00620217.xml
./new_data/combined_xml/NCT01392209.xml
./new_data/combined_xml/NCT03384212.xml
./new_data/combined_xml/NCT02559115.xml
./new_data/combined_xml/NCT02079740.xml
./new_data/combined_xml/NCT02413008.xml
./new_data/combined_xml/NCT03632343.xml
./new_data/combined_xml/NCT01260935.xml
./new_data/combined_xml/NCT01605396.xml
./new_data/combined_xml/NCT02831634.xml
./new_data/combined_xml/NCT03271099.xml
./new_data/combined_xml/NCT02103946.xml
./new_data/combined_xml/NCT00655850.xml


./new_data/combined_xml/NCT03238261.xml
./new_data/combined_xml/NCT00963859.xml
./new_data/combined_xml/NCT00903175.xml
./new_data/combined_xml/NCT03521219.xml
./new_data/combined_xml/NCT00794638.xml
./new_data/combined_xml/NCT00653367.xml
./new_data/combined_xml/NCT02397967.xml
./new_data/combined_xml/NCT02730091.xml
./new_data/combined_xml/NCT01191632.xml
./new_data/combined_xml/NCT03538990.xml
./new_data/combined_xml/NCT02229045.xml
./new_data/combined_xml/NCT00455962.xml
./new_data/combined_xml/NCT02859402.xml
./new_data/combined_xml/NCT00613743.xml
./new_data/combined_xml/NCT02374424.xml
./new_data/combined_xml/NCT02475213.xml
./new_data/combined_xml/NCT01313689.xml
./new_data/combined_xml/NCT03155009.xml
./new_data/combined_xml/NCT03135925.xml
./new_data/combined_xml/NCT03793361.xml
./new_data/combined_xml/NCT02332317.xml
./new_data/combined_xml/NCT00070083.xml
./new_data/combined_xml/NCT00746018.xml
./new_data/combined_xml/NCT02051634.xml
./new_data/combined_xml/NCT00726934.xml


In [10]:
xml_file = "./new_data/combined_xml/NCT00000500.xml"
tree = et.parse(xml_file)
root = tree.getroot()

#res = []
#for t in root: 
#col_names = 
# name1 = ['nct_id', 'official_title', 'overall_status', 
#          'start_date','completion_date', 'phase', 'study_type']
name2 = ['brief_summary', 'detailed_description', 'intervention', 
         'condition', 'enrollment', 'eligibility']
df = pd.DataFrame(columns = name2)
row = {name: ''for name in name2} 
for attribute in root:
    print(attribute)
# for d in tree.iter():
#    if d.tag =='condition':
#     print([txt.text for txt in d.iter()])
    
#        if d.tag =='brief_summary':
#     print(d.getchildren()[0].text)
    
# df = df.append(pd.Series(row), ignore_index = True)
# df
#return df    
#     elif d.tag in name2:
#         print d.attrib.values()[0]

# for child in root:
#     #print(child.tag, child.attrib, child.text)
#     if child.tag == 'id_info':
#         for c in child:
#             print(c.tag, c.attrib, c.text)

<Element 'required_header' at 0x12353c818>
<Element 'id_info' at 0x1238b7688>
<Element 'brief_title' at 0x1238b7cc8>
<Element 'sponsors' at 0x1238b7ea8>
<Element 'source' at 0x1238b7ae8>
<Element 'brief_summary' at 0x1238b7818>
<Element 'detailed_description' at 0x1238b77c8>
<Element 'overall_status' at 0x1238b7368>
<Element 'start_date' at 0x1238b7638>
<Element 'completion_date' at 0x1238b7548>
<Element 'primary_completion_date' at 0x11f2f7688>
<Element 'phase' at 0x11f2f7958>
<Element 'study_type' at 0x11f2f7548>
<Element 'has_expanded_access' at 0x12398dd68>
<Element 'study_design_info' at 0x12398dd18>
<Element 'condition' at 0x12398d4f8>
<Element 'condition' at 0x12393b458>
<Element 'condition' at 0x12393b548>
<Element 'condition' at 0x12393b6d8>
<Element 'intervention' at 0x12393b8b8>
<Element 'intervention' at 0x12393be58>
<Element 'eligibility' at 0x12393b818>
<Element 'reference' at 0x12393b2c8>
<Element 'reference' at 0x12158cef8>
<Element 'reference' at 0x122c9d9f8>
<Element 

In [11]:
df

,brief_summary,detailed_description,intervention,condition,enrollment,eligibility


In [12]:
xml_file = "./data/search_result/NCT00000479.xml"

In [13]:
def parse_XML(xml_file): 
    """Parse the input XML file and store the result 
    in a pandas DataFrame with the given columns"""
    tree = et.parse(xml_file)
    root = tree.getroot()
    text = {}
    df_cols = ['eligibility', 'intervention_name', 'condition', 'enrollment']
    
    df = pd.DataFrame(columns = df_cols)
    
    try:
        for  tag in tags:
            if tag == "eligibility" :
                text[tag]= root.find('eligibility').find('criteria').find('textblock').text
            if tag ==  "intervention_name":
                intBrowse = root.find('intervention_browse')
                if  intBrowse:
                    text[tag]= [i.text for i in intBrowse.findall('mesh_term') ]
                else:
                    text[tag] = [i.text for i in root.iter('intervention_name') ]
            if tag == "condition" :
                text[tag]=  [c.text for c in root.findall('condition') ]
            if tag == "enrollment":
                text[tag]= root.find('enrollment').text
    except:
        print ("no eligibility criteria")

    df = df.append(pd.Series(text, index = df_cols), ignore_index = True)

    return df

In [14]:
parse_XML(xml_file)

no eligibility criteria


,eligibility,intervention_name,condition,enrollment
0,NaN,NaN,NaN,NaN


In [15]:
def parse_XML(xml_file): 
    """Parse the input XML file and store the result 
    in a pandas DataFrame with the given columns"""
    tree = et.parse(xml_file)
    root = tree.getroot()
    
#     df_cols = ['clinical_study', 'nct_id', 'official_title',
#                'brief_summary', 'detailed_description', 'overall_status', 
#                'start_date','completion_date', 'phase', 'study_type', 
#                'study_design_info','eligibility','intervention', 'condition', 
#                'enrollment']

    df_cols = ['clinical_study', 'nct_id', 'official_title']
    
    df = pd.DataFrame(columns = df_cols)

    for node in root: 
        clinical_study = node.attrib.get('rank')
        nct_id = node.find('nct_id')
        official_title = node.find('official_title')
#         brief_summary = node.find('brief_summary')
#         detailed_description = node.find('detailed_description')
#         overall_status = node.find('overall_status')
#         start_date = node.attrib.get('start_date')
#         completion_date = node.attrib.get('completion_date')
#         phase = node.attrib.get('phase')
#         study_type = node.attrib.get('study_type')
#         study_design_info = node.attrib.get('study_design_info')
#         #eligibility = node.find('eligibility').find('criteria').find('textblock').text
#         intervention = node.find('intervention_browse')
#         condition = node.findall('condition')
#         #enrollment = node.find('enrollment').text

        df = df.append(pd.Series([clinical_study, nct_id, official_title], index = df_cols), ignore_index = True)

    return df

In [16]:
file_path = "./data/search_result/NCT00000479.xml"

parse_XML(file_path)

,clinical_study,nct_id,official_title
0,None,None,None
1,None,[],None
2,None,None,None
3,None,None,None
4,None,None,None
5,None,None,None
6,None,None,None
7,None,None,None
8,None,None,None
9,None,None,None


In [17]:
for child in root:
    print(child.tag, child.attrib)

required_header {}
id_info {}
brief_title {}
sponsors {}
source {}
brief_summary {}
detailed_description {}
overall_status {}
start_date {}
completion_date {'type': 'Actual'}
primary_completion_date {'type': 'Actual'}
phase {}
study_type {}
has_expanded_access {}
study_design_info {}
condition {}
condition {}
condition {}
condition {}
intervention {}
intervention {}
eligibility {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
refer

In [18]:
from xmljson import badgerfish as bf
from xml.etree.ElementTree import fromstring

In [19]:
!python -m xmljson -d badgerfish ./data/search_result/NCT00000479.xml

{
  "clinical_study": {
    "@rank": 5313,
    "required_header": {
      "download_date": {
        "$": "ClinicalTrials.gov processed this data on April 29, 2019"
      },
      "link_text": {
        "$": "Link to the current ClinicalTrials.gov record."
      },
      "url": {
        "$": "https://clinicaltrials.gov/show/NCT00000479"
      }
    },
    "id_info": {
      "org_study_id": {
        "$": 69
      },
      "secondary_id": [
        {
          "$": "R01HL043851"
        },
        {
          "$": "HL043851"
        },
        {
          "$": "CA047988"
        }
      ],
      "nct_id": {
        "$": "NCT00000479"
      }
    },
    "brief_title": {
      "$": "Women's Health Study (WHS): A Randomized Trial of Low-dose Aspirin and Vitamin E in the Primary Prevention of Cardiovascular Disease and Cancer"
    },
    "acronym": {
      "$": "WHS"
    },
    "official_title": {
      "$": "Women's Health Study of Low-dose Aspirin and Vitamin E in Apparently Healthy Wome

                  }
                }
              }
            ]
          }
        }
      },
      "certain_agreements": {
        "pi_employee": {
          "$": "All Principal Investigators ARE employed by the organization sponsoring the study."
        },
        "restrictive_agreement": {
          "$": "There is NOT an agreement between Principal Investigators and the Sponsor (or its agents) that restricts the PI's rights to discuss or publish trial results after the trial is completed."
        }
      },
      "point_of_contact": {
        "name_or_title": {
          "$": "Julie E. Buring"
        },
        "organization": {
          "$": "Brigham and Women's Hospital"
        },
        "phone": {
          "$": "617 732 4965"
        },
        "email": {
          "$": "jburing@rics.bwh.harvard.edu"
        }
      }
    }
  }
}

In [20]:
!for f in $(ls | grep '.xml'); do python -m xmljson -d yahoo $f > $json/f.json ; done

In [21]:
df.head()

,brief_summary,detailed_description,intervention,condition,enrollment,eligibility


In [22]:
for child in root:
    print(child.tag, child.attrib)

required_header {}
id_info {}
brief_title {}
sponsors {}
source {}
brief_summary {}
detailed_description {}
overall_status {}
start_date {}
completion_date {'type': 'Actual'}
primary_completion_date {'type': 'Actual'}
phase {}
study_type {}
has_expanded_access {}
study_design_info {}
condition {}
condition {}
condition {}
condition {}
intervention {}
intervention {}
eligibility {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
reference {}
refer

In [23]:
dfcols = ['clinical_study', 'nct_id', 'brief_title']
df = pd.DataFrame(columns=dfcols)

for i in etree.iter(tag='data'):
df = df.append(
        pd.Series([i.get('id'), i.get('name')], index=dfcols),
        ignore_index=True)
df.head()

IndentationError: expected an indented block (<ipython-input-23-55b1bc950e52>, line 5)

In [ ]:
trials.head()

In [ ]:
# Retrieve Tweets
MaxTweets = 10000000
tweetsPerQry = 100
fName = 'tweets.txt' # Storage name

tweetCount = 0
print("Downloading max {0} tweets".format(MaxTweets))
with open(fName, 'w') as f:
    for tweet in tweepy.Cursor(api.search,q=query, wait_on_rate_limit=True).items(MaxTweets) :         

        # Verify the tweet has place info before writing
        if tweet.place is not None:
            
            #Write to the text file, and add one to the number of tweets
            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
            tweetCount += 1

    #Display how many tweets we have collected
    print("Downloaded {0} tweets".format(tweetCount))
    

In [ ]:
# Checking the data
tweet._json

In [ ]:
max_id = -1
tweetCount = 0
with open('PoGo_USA_Tutorial.json', 'w') as f:
    #While we still want to collect more tweets
    while tweetCount < MaxTweets:
        try:
            #Look for more tweets, resuming where we left off
            if max_id <= 0:
                new_tweets = api.search(q=query, count=tweetsPerQry)
            else:
                new_tweets = api.search(q=query, count=tweetsPerQry, max_id=str(max_id - 1))
            
            #If we didn't find any exit the loop
            if not new_tweets:
                print("No more tweets found")
                break
            
            #Write the JSON output of any new tweets we found to the output file
            for tweet in new_tweets:
                
                #Make sure the tweet has place info before writing
                if (tweet.place is not None) and (tweetCount < MaxTweets):
                    f.write(jsonpickle.encode(tweet._json, unpicklable=False) +
                        '\n')
                    tweetCount += 1
                    
            #Display how many tweets we have collected
            print("Downloaded {0} tweets".format(tweetCount))
            
            #Record the id of the last tweet we looked at
            max_id = new_tweets[-1].id
            
        except tweepy.TweepError as e:
            
            #Print the error and continue searching
            print("some error : " + str(e))


print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

In [ ]:
#search for multiple phrases using OR (for Pharma companies)
Allquery = "'Johnson & Johnson' or 'J & J' or 'J&J' or 'JNJ' or '$JNJ' \
        or 'Pfizer' or 'pfizer' or 'PFE' or '$PFE' \
        or 'Novartis' or 'novartis' or 'NOVN' or '$NOVN' \
        or 'Roche' or 'roche' or 'ROG' or '$ROG' \
        or 'Merck & Co.' or 'Merck' or 'MRK' or '$MRK' \
        or 'AbbVie' or 'abbvie' or 'ABBV' or '$ABBV' \
        or 'Eli Lilly' or 'EliLilly' or 'LLY' or '$LLY' \
        or 'Sanofi' or 'sanofi' or 'SAN' or '$SAN' \
        or 'AstraZeneca' or 'astrazeneca' or 'AZN' or '$AZN' \
        or 'GlaxoSmithKline' or 'GSK' or '$GSK'"